Marcus Blaisdell
CptS 437
Homework #4
March 5, 2019
Professor Cook

1)

Mean Squared Error:
For each point x, calculate prediction, Y_hat by: Y_hat=Theta_0 + Theta_1 * x_i (see linked spreadsheet)
Sum all of the Squares of the difference of y and Y_hat, and divide by total number of samples (20)
MSE = 18992.12598

https://docs.google.com/spreadsheets/d/1sHtyHo-NNmPwknHD-8RsrxPwf1vhQSIzuAEsI0KeNso/edit?usp=sharing

Update Theta_0 and Theta_1 by:
θ_0=θ_0-α(1/2n ∑_(i=1)^n▒〖(2θ_0+2θ_1 x_i 〗-2y_i))
θ_1=θ_1-α(1/2n ∑_(i=1)^n▒〖(2θ_0 x_i 〗-2x_i y_i+〖2θ_1 x〗_i^2))

https://drive.google.com/file/d/1_C2jjt7wGNilIOWSzNBYLlLxyxLAbLpg/view?usp=sharing

New θ_0 = 7.441607029
New θ_1 = 0.003059615741


2)

The t-value calculates as 3.06. 
For degrees-of-freedom = 6, p-value is between 2.447 and 3.143 which is between 95% and 98% confidence.
As the confidence is greater than 95%, the diffrerence in f-measures for Algorithm 1 and Algorithm 2 are statistically significant. 

(See link for work)

https://docs.google.com/spreadsheets/d/12h75IdNeEiC_HbhcM-m9_Ql13-1W1j5JNOqUkg3Es_Q/edit?usp=sharing


3)

From the "Machine Learning Crash Course" online tutorial at:
https://developers.google.com/machine-learning/crash-course/regularization-for-simplicity/lambda

as lambda increases (tends towards infinity), the model will become more simple. This  would effectively be a higher bias and a lower variance, and as lambda decreases (tends towards zero), the bias will decrease and the variance will increase as the model becomes more complex.

The lower lambda value will increase the risk of overfitting the data whereas higher lambda values will have a risk of underfitting the data.



4)

Our chosen project is stock market value prediction for a single company (Microsoft).

Our team has two members: Ngoc Duong and Marcus Blaisdell. We will work on every part of this project together. Thus, there is no specific role for each of us.

The data that this project will use is a time-series. There are two possible approaches to this project: a Recurrent Neural Networks (RNN) and a Long, Short-Term Memory Neural Network (LSTM NN). After doing some research, we found that a RNN can only look back to several time steps while a LSTM NN can look back to many time steps. Considering that with the nature of stock market, we believe that a LSTM NN will be the best method to solve this problem.

The datasets that we will use will be daily stock information from the past ten years for Microsoft and related companies. We are obtaining our data from NASDAQ.com (National Association of Securities Dealers Automated Quotations).  It is our premise that the stock market value will be affected by other companies that Microsoft works with, and competes against. For example, Intel processors, Dell computers, Apple computers, cloud computing hardware manufacturers. This is a sample list and not comprehensive of the companies we will consider in our project. By analyzing related companies simultaneously, we seek to improve the ability of the ML algorithm to provide a good prediction.

We will evaluate the accuracy of the predictions using a variety of time sequences, for example, varying lengths of historical data for training. The performance will be measured using the four main methods of: Accuracy, Precision, Recall, and f1 measure. 

The feature set will be stock prices and other stock information of Microsoft in the past ten years and a combination of the related companies. We will experiment with combining different companies to test if adding their data improves the prediction of the price of the target company. For example, to compare the history of Intel processors affect on Microsoft prices, the feature set could be: 
{Microsoft_open, Microsoft_high, Microsoft_low, Microsoft_close, Microsoft_volume, Intel_open, Intel_high, Intel_low, Intel_close, Intel_volume}

We will also experiment with adding the weekly, monthly, 3-month averages as a feature to determine if it proves useful as a predictive feature. 

Working on this project, we hope to learn the basic of stock investment, particularly how stock price is decided or can be predicted. We also want to experience learning a machine learning method (long short term memory approach) by ourselves (without class lectures) and applying what we learnt in a real-world problem. In additional, lately, we have come to know that our chosen project is a different type of machine learning problem (sequential patterns/time series) than what we have learnt in class (classifiers and regressors). Thus, we are excited to explore this new type of machine learning problems and approaches.

This information could be useful to experts in both the target company and investment firms. The target company could use the information to adjust their company strategies based on key market indicators. Stock market investors can use this information to adjust their trading plans to optimize their returns. 


5)



In [34]:
import numpy as np
import copy
import math
from sklearn.naive_bayes import GaussianNB
from google.colab import drive
drive.mount('/content/gdrive')

#########################################################
# formatData function:
# the data is arranged with the labels in the 
# last position and the features in all of the 
# preceding positions
# This function reads the data
# and returns the attributes and labels
# as separate arrays:


def formatData (Data):
  X = []
  y = []

  for i in range (len(Data)):
    y.append (Data[i][-1])
    X.append (Data[i][:-1])

  return X, y

# end formatData function
#########################################################

#########################################################
# myKFolds function:
# divides the list into K, mostly equal parts
# and returns a list with all of the indexes

def myKFolds (theSize, theK):
  theList = []
  spacing = int (theSize / theK)

  for i in range (theK - 1):
    theList.append([i * spacing, i * spacing + spacing - 1])

  theList.append([(theK - 1) * spacing, theSize])

  return theList

#
#########################################################


#########################################################
# discretize function:
# divides the list into 10, equal parts
# and returns a list with all of the bucket ranges

def discretize (theVal):
  if theVal == [0]:
    return [0]
  else:
    theBuckets = []
    theSize = theVal / 10

    for i in range (9):
      theBuckets.append([i * theSize, i * theSize + theSize])

    theBuckets.append([9 * theSize, theVal])
    
    return theBuckets

#
#########################################################


#########################################################
# modData function:
# Finds the max value of each attribute
# and converts each value to a discrete, normalized 
# value between 0 and 9

def modData (Data):
    # build a list of attributes:

  attributesList = []
  
    # add a list for each attribute
    # we will add the ten buckets to this:
    # but first, use it to store the max value:
    # exclude the last element, it is the label, 
    # we don't want to modify it:
    
  for i in range (len(Data[0]) - 1):
    attributesList.append([0])
    
  # find the max value for each attribute:
  # requires reading the entire list through once:
  
  for i in range (len(Data)):
    for j in range (len(Data[0]) - 1):
      if Data[i][j] > attributesList[j]:
        attributesList[j] = Data[i][j]
        
  # Now that we have all the max values,
  # discretize them and add our buckets in place of our max value:
  
  for i in range (len(attributesList)):
    attributesList[i] = discretize (attributesList[i])
      
    
  # Update all values in original Data with new discretized values:
  #print ("len(Data): ", len(Data))
  for i in range (len(Data)):
    for j in range (len(attributesList)):
      if len(attributesList[j]) == 1:
        Data[i][j] = 0
        #print ("newData[i][j]: 0\n")
      else:
        for k in range (10):
          #print ("Data[i][j]: ", Data[i][j])
          #print ("attributesList[j][k][0]: ", attributesList[j][k][0])
          #print ("attributesList[j][k][1]: ", attributesList[j][k][1])
          if Data[i][j] == 0:
            Data[i][j] = 0
            #print ("newData[i][j]: 0\n")
          if (Data[i][j] > attributesList[j][k][0]) and (Data[i][j] <= attributesList[j][k][1]):
            Data[i][j] = k
            #print ("new Data[i][j]: ", k, "\n")
          
  return Data
      

#
#########################################################


#########################################################
# t_test function
# calculate the t-value of two sets of scores
# to test if they are statistically significant

def t_test (aVals, bVals):
  
    # get the means of each set:
  
  aMean = getMean (aVals)
  bMean = getMean (bVals)
  
    # get the summation of the deviations:
    
  summation = getSummation (aVals, bVals, aMean, bMean)
  
    # N is the number of samples:
  
  N = len(aVals)
  
    # Calculate the t-value:
  
  tValue = (aMean - bMean) * math.sqrt( ((N*(N-1))/summation) )
  
  return tValue
  

#
#########################################################



#########################################################
# getMean function
# calculates and returns the average value of a list

def getMean (theList):
  theSum = 0
  
  for value in theList:
    theSum += value
    
  theSum = float (theSum) / len(theList)
  
  return theSum

#
#########################################################


#########################################################
# getSummation function
# calculates and returns the sum of the deviations

def getSummation (aVals, bVals, aMean, bMean):
  runSum = 0
  
  for i in range (len(aVals)):
    runSum += ((aVals[i] - aMean) - (bVals[i] - bMean))**2
    
  return runSum

#
#########################################################



#########################################################
# Main function:

if __name__ == "__main__":
  
  
    # create three lists to hold the scores from each function:
  
  A = []
  B = []
  C = []
  
  
      # Read data from file:
    
  Data = np.loadtxt(fname='/content/gdrive/My Drive/437/HW4/smarthome.csv', delimiter=',')
  #Data = np.loadtxt(fname='/content/gdrive/My Drive/437/HW4/smarthome-medium.csv', delimiter=',')
  #Data = np.loadtxt(fname='/content/gdrive/My Drive/437/HW4/smarthome-small.csv', delimiter=',')


  print ("Part 1:")
  
  
    # format the data:
    
  X, y = formatData (Data)

  clf = GaussianNB()
  
    # use 10-folds:
  
  numFolds = 10
  
  myList = myKFolds (len(X), numFolds)
  
    # Traverse the folds:
  
  for i in range (len(myList)):
  
    testData = X[myList[i][0]:myList[i][1] + 1]
    testLabel = y[myList[i][0]:myList[i][1] + 1]
    
      # initialize trainset with i + 1 % len(list)
      # to get the next index in a circular array:
  
    trainData = X[myList[(i + 1) % len(myList)][0]:myList[(i + 1) % len(myList)][1] + 1]
    trainLabel = y[myList[(i + 1) % len(myList)][0]:myList[(i + 1) % len(myList)][1] + 1]
    
      # now, concatenate remaining indexes that are not the test index
    
    for j in range (1, len(myList) - 1):

      trainData = np.concatenate ((trainData, X[myList[(i + 1 + j) % len(myList)][0]:myList[(i + 1 + j) % len(myList)][1] + 1]), axis=0)
      trainLabel = np.concatenate ((trainLabel, y[myList[(i + 1 + j) % len(myList)][0]:myList[(i + 1 + j) % len(myList)][1] + 1]), axis=0)

      # train the model:
      
    clf.fit(trainData,trainLabel)

      # test the model:
      
    correct = 0

    for testIndex in range (len(testData)):
      #print ("clf.predict_proba ( [testData[testIndex]]) ): ", clf.predict_proba ( [testData[testIndex]]) ) 
      if (clf.predict ( [testData[testIndex]]) ) == testLabel[testIndex]:
        correct += 1

    theAccuracy = 100 * (float (correct) / len(testData) )
    
    print ("Accuracy: ", theAccuracy )
    
    A.append (theAccuracy)
    
  
  ### End part 1
  ########################################################################
  
  print ("Part 2:")

  Data = modData (Data)
  
  #print ("Data[0]: ", Data[0])
  
      # format the data:
    
  X, y = formatData (Data)

  clf = GaussianNB()
  
    # use 10-folds:
  
  numFolds = 10
  
  myList = myKFolds (len(X), numFolds)
  
    # Traverse the folds:
  
  for i in range (len(myList)):
  
    testData = X[myList[i][0]:myList[i][1] + 1]
    testLabel = y[myList[i][0]:myList[i][1] + 1]
    
      # initialize trainset with i + 1 % len(list)
      # to get the next index in a circular array:
  
    trainData = X[myList[(i + 1) % len(myList)][0]:myList[(i + 1) % len(myList)][1] + 1]
    trainLabel = y[myList[(i + 1) % len(myList)][0]:myList[(i + 1) % len(myList)][1] + 1]
    
      # now, concatenate remaining indexes that are not the test index
    
    for j in range (1, len(myList) - 1):

      trainData = np.concatenate ((trainData, X[myList[(i + 1 + j) % len(myList)][0]:myList[(i + 1 + j) % len(myList)][1] + 1]), axis=0)
      trainLabel = np.concatenate ((trainLabel, y[myList[(i + 1 + j) % len(myList)][0]:myList[(i + 1 + j) % len(myList)][1] + 1]), axis=0)

      # train the model:
      
    clf.fit(trainData,trainLabel)

      # test the model:
      
    correct = 0

    for testIndex in range (len(testData)):
      if (clf.predict ( [testData[testIndex]]) ) == testLabel[testIndex]:
        correct += 1

    theAccuracy = 100 * (float (correct) / len(testData) )
    print ("Accuracy: ",  theAccuracy)
    
    B.append (theAccuracy)
    
  
  ### End part 2
  ########################################################################
  
  
  print ("Part 3:")
  
  
    # create an integer to hold the prediction from the 8 models
    
  thePrediction = 0
  
    # format the data:
    
  X, y = formatData (Data)

  clf_0 = GaussianNB()
  clf_1 = GaussianNB()
  clf_2 = GaussianNB()
  clf_3 = GaussianNB()
  clf_4 = GaussianNB()
  clf_5 = GaussianNB()
  clf_6 = GaussianNB()
  clf_7 = GaussianNB()
  
    # use 10-folds:
  
  numFolds = 10
  
  myList = myKFolds (len(X), numFolds)
  
    # Traverse the folds:
  
  for i in range (len(myList)):
  
    testData = X[myList[i][0]:myList[i][1] + 1]
    testLabel = y[myList[i][0]:myList[i][1] + 1]
    
      # initialize trainset with i + 1 % len(list)
      # to get the next index in a circular array:
  
    trainData = X[myList[(i + 1) % len(myList)][0]:myList[(i + 1) % len(myList)][1] + 1]
    trainLabel = y[myList[(i + 1) % len(myList)][0]:myList[(i + 1) % len(myList)][1] + 1]
    
      # now, concatenate remaining indexes that are not the test index
    
    for j in range (1, len(myList) - 1):

      trainData = np.concatenate ((trainData, X[myList[(i + 1 + j) % len(myList)][0]:myList[(i + 1 + j) % len(myList)][1] + 1]), axis=0)
      trainLabel = np.concatenate ((trainLabel, y[myList[(i + 1 + j) % len(myList)][0]:myList[(i + 1 + j) % len(myList)][1] + 1]), axis=0)

    
    trainLabel_0 = copy.deepcopy(trainLabel)
    trainLabel_1 = copy.deepcopy(trainLabel)
    trainLabel_2 = copy.deepcopy(trainLabel)
    trainLabel_3 = copy.deepcopy(trainLabel)
    trainLabel_4 = copy.deepcopy(trainLabel)
    trainLabel_5 = copy.deepcopy(trainLabel)
    trainLabel_6 = copy.deepcopy(trainLabel)
    trainLabel_7 = copy.deepcopy(trainLabel)
    
    testLabel_0 = copy.deepcopy(testLabel)
    testLabel_1 = copy.deepcopy(testLabel)
    testLabel_2 = copy.deepcopy(testLabel)
    testLabel_3 = copy.deepcopy(testLabel)
    testLabel_4 = copy.deepcopy(testLabel)
    testLabel_5 = copy.deepcopy(testLabel)
    testLabel_6 = copy.deepcopy(testLabel)
    testLabel_7 = copy.deepcopy(testLabel)
    
    # convert the labels for one-versus-all:
    
    for index in range(len(trainLabel_0)):
      if trainLabel_0[index] != 0:
        trainLabel_0[index] = 9
        
    for index in range(len(trainLabel_1)):
      if trainLabel_1[index] != 1:
        trainLabel_1[index] = 9
            
    for index in range(len(trainLabel_2)):
      if trainLabel_2[index] != 2:
        trainLabel_2[index] = 9
        
    for index in range(len(trainLabel_3)):
      if trainLabel_3[index] != 3:
        trainLabel_3[index] = 9
        
    for index in range(len(trainLabel_4)):
      if trainLabel_4[index] != 4:
        trainLabel_4[index] = 9
        
    for index in range(len(trainLabel_5)):
      if trainLabel_5[index] != 5:
        trainLabel_5[index] = 9
        
    for index in range(len(trainLabel_6)):
      if trainLabel_6[index] != 6:
        trainLabel_6[index] = 9      
        
    for index in range(len(trainLabel_7)):
      if trainLabel_7[index] != 7:
        trainLabel_7[index] = 9
        
    for index in range(len(testLabel_0)):
      if testLabel_0[index] != 0:
        testLabel_0[index] = 9
        
    for index in range(len(testLabel_1)):
      if testLabel_1[index] != 1:
        testLabel_1[index] = 9        
        
    for index in range(len(testLabel_2)):
      if testLabel_2[index] != 2:
        testLabel_2[index] = 9
        
    for index in range(len(testLabel_3)):
      if testLabel_3[index] != 3:
        testLabel_3[index] = 9
        
    for index in range(len(testLabel_4)):
      if testLabel_4[index] != 4:
        testLabel_4[index] = 9
        
    for index in range(len(testLabel_5)):
      if testLabel_5[index] != 5:
        testLabel_5[index] = 9
        
    for index in range(len(testLabel_6)):
      if testLabel_6[index] != 6:
        testLabel_6[index] = 9
        
    for index in range(len(testLabel_7)):
      if testLabel_7[index] != 7:
        testLabel_7[index] = 9
        
        # train the model:
      
    clf_0.fit(trainData,trainLabel_0)
    clf_1.fit(trainData,trainLabel_1)
    clf_2.fit(trainData,trainLabel_2)
    clf_3.fit(trainData,trainLabel_3)
    clf_4.fit(trainData,trainLabel_4)
    clf_5.fit(trainData,trainLabel_5)
    clf_6.fit(trainData,trainLabel_6)
    clf_7.fit(trainData,trainLabel_7)

      # test the model:
      
    correct = 0

    for testIndex in range (len(testData)):
      #pred_0 = clf_0.predict_proba ( [testData[testIndex]])[0][0]
      #print ("showVal: ", pred_0, " - testLabel_0[testIndex]: ", testLabel_0[testIndex])
        
        # get max of all 8 scores for this point:
        # set maxScoreIndex to zero initially,
        # if a higher value is found from any of the remaining 7 scores,
        # replace the max score with that score,
        # and update the predicted value, thePrediction
        
      score_0 = clf_0.predict_proba ( [testData[testIndex]])[0][0]
      maxScore = score_0
      thePrediction = 0
      
      score_1 = clf_1.predict_proba ( [testData[testIndex]])[0][0]
      if score_1 > maxScore:
        maxScore = score_1
        thePrediction = 1
        
      score_2 = clf_2.predict_proba ( [testData[testIndex]])[0][0]
      if score_2 > maxScore:
        maxScore = score_2
        thePrediction = 2
      
      score_3 = clf_3.predict_proba ( [testData[testIndex]])[0][0]
      if score_3 > maxScore:
        maxScore = score_3
        thePrediction = 3
        
      score_4 = clf_4.predict_proba ( [testData[testIndex]])[0][0]
      if score_4 > maxScore:
        maxScore = score_4
        thePrediction = 4
        
      score_5 = clf_5.predict_proba ( [testData[testIndex]])[0][0]
      if score_5 > maxScore:
        maxScore = score_5
        thePrediction = 5
        
      score_6 = clf_6.predict_proba ( [testData[testIndex]])[0][0]
      if score_6 > maxScore:
        maxScore = score_6
        thePrediction = 6
        
      score_7 = clf_7.predict_proba ( [testData[testIndex]])[0][0]
      if score_7 > maxScore:
        maxScore = score_7
        thePrediction = 7
      
      if thePrediction == testLabel[testIndex]:
        correct += 1

    theAccuracy = 100 * (float (correct) / len(testData) )
    
    print ("Accuracy: ", theAccuracy )
    
    C.append(theAccuracy)
    
    
  ### End part 3
  ########################################################################

    
  tScoreAB = t_test (A, B)
  tScoreAC = t_test (A, C)
  tScoreBC = t_test (B, C)
  
  print ("\n\nFor Degrees-of-Freedom = 9, a t-value greater than 2.262")
  print ("has a confidence level greater than 95%")
  print ("and is therefor Statistically significant\n\n")
  
  print ("\n\nt-score for Part 1 and Part 2: ", tScoreAB )
  if tScoreAB > 2.262:
    print ("\tStatistically Significant")
  else:
    print ("\tnot Statistically Significant")
  print ("\n\nt-score for Part 1 and Part 3: ", tScoreAC )
  if tScoreAC > 2.262:
    print ("\tStatistically Significant")
  else:
    print ("\tnot Statistically Significant")
  print ("\n\nt-score for Part 2 and Part 3: ", tScoreBC )
  if tScoreBC > 2.262:
    print ("\tStatistically Significant")
  else:
    print ("\tnot Statistically Significant")
  
# end Main function  
#########################################################
  

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Part 1:
Accuracy:  84.0054548996688
Accuracy:  83.81063705435417
Accuracy:  78.92070913695694
Accuracy:  81.80401324761348
Accuracy:  85.36917981687122
Accuracy:  84.9600623417105
Accuracy:  86.92772257938827
Accuracy:  88.4473017728424
Accuracy:  83.86908240794857
Accuracy:  90.9727626459144
Part 2:
Accuracy:  81.97934930839665
Accuracy:  80.32339762322229
Accuracy:  74.34248977206312
Accuracy:  75.02435223066432
Accuracy:  78.97915449055134
Accuracy:  78.0440288330411
Accuracy:  82.99240210403272
Accuracy:  72.41379310344827
Accuracy:  72.86187414767193
Accuracy:  73.59922178988327
Part 3:
Accuracy:  78.53107344632768
Accuracy:  76.6218585622443
Accuracy:  69.37463471654004
Accuracy:  73.2904734073641
Accuracy:  78.43366452367036
Accuracy:  79.25189947399181
Accuracy:  78.08299240210404
Accuracy:  77.40112994350282
Accuracy:  65.67309565556205
Accuracy:  